In [1]:
# Simple Lin Reg Approach

In [2]:
import polars as pl
import numpy as np
from sklearn import preprocessing, linear_model, model_selection, pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR

In [3]:
# data handling
input_paths = 'inputs/train.parquet/*/*.parquet'
scan_df = pl.scan_parquet(input_paths)

sample_df = scan_df.head(int(1e4)).collect()
sample_df


date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,…,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,partition_id
i16,i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i64
0,0,1,3.889038,null,null,null,null,null,0.851033,0.242971,0.2634,-0.891687,11,7,76,-0.883028,0.003067,-0.744703,null,-0.169586,null,-1.335938,-1.707803,0.91013,null,1.636431,1.522133,-1.551398,-0.229627,null,null,1.378301,-0.283712,0.123196,null,null,…,null,null,-0.808103,null,-2.037683,0.727661,null,-0.989118,-0.345213,-1.36224,null,null,null,null,null,-1.251104,-0.110252,-0.491157,-1.02269,0.152241,-0.659864,null,null,-0.261412,-0.211486,-0.335556,-0.281498,0.738489,-0.069556,1.380875,2.005353,0.186018,1.218368,0.775981,0.346999,0.095504,0
0,0,7,1.370613,null,null,null,null,null,0.676961,0.151984,0.192465,-0.521729,11,7,76,-0.865307,-0.225629,-0.582163,null,0.317467,null,-1.250016,-1.682929,1.412757,null,0.520378,0.744132,-0.788658,0.641776,null,null,0.2272,0.580907,1.128879,null,null,…,null,null,-1.625862,null,-1.410017,1.063013,null,0.888355,0.467994,-1.36224,null,null,null,null,null,-1.065759,0.013322,-0.592855,-1.052685,-0.393726,-0.741603,null,null,-0.281207,-0.182894,-0.245565,-0.302441,2.965889,1.190077,-0.523998,3.849921,2.626981,5.0,0.703665,0.216683,0.778639,0
0,0,9,2.285698,null,null,null,null,null,1.056285,0.187227,0.249901,-0.77305,11,7,76,-0.675719,-0.199404,-0.586798,null,-0.814909,null,-1.296782,-2.040234,0.639589,null,1.597359,0.657514,-1.350148,0.364215,null,null,-0.017751,-0.317361,-0.122379,null,null,…,null,null,-0.72542,null,-2.29417,1.764551,null,-0.120789,-0.063458,-1.36224,null,null,null,null,null,-0.882604,-0.072482,-0.617934,-0.86323,-0.241892,-0.709919,null,null,0.377131,0.300724,-0.106842,-0.096792,-0.864488,-0.280303,-0.326697,0.375781,1.271291,0.099793,2.109352,0.670881,0.772828,0
0,0,10,0.690606,null,null,null,null,null,1.139366,0.273328,0.306549,-1.262223,42,5,150,-0.694008,3.004091,0.114809,null,-0.251882,null,-1.902009,-0.979447,0.241165,null,-0.392359,-0.224699,-2.129397,-0.855287,null,null,0.404142,-0.578156,0.105702,null,null,…,null,null,1.313203,null,-0.810125,2.939022,null,3.988801,1.834661,-1.36224,null,null,null,null,null,-0.697595,1.074309,-0.206929,-0.530602,4.765215,0.571554,null,null,-0.226891,-0.251412,-0.215522,-0.296244,0.408499,0.223992,2.294888,1.097444,1.225872,1.225376,1.114137,0.775199,-1.379516,0
0,0,14,0.44057,null,null,null,null,null,0.9552,0.262404,0.344457,-0.613813,44,3,16,-0.947351,-0.030018,-0.502379,null,0.646086,null,-1.844685,-1.58656,-0.182024,null,-0.969949,-0.673813,-1.282132,-1.399894,null,null,0.043815,-0.320225,-0.031713,null,null,…,null,null,0.476195,null,-0.771732,2.843421,null,1.379815,0.411827,-1.36224,null,null,null,null,null,-0.948601,-0.136814,-0.447704,-1.141761,0.099631,-0.661928,null,null,3.678076,2.793581,2.61825,3.418133,-0.373387,-0.502764,-0.348021,-3.928148,-1.591366,-5.0,-3.57282,-1.089123,-5.0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,266,3

In [4]:
# Specify columns to drop
columns = sample_df.columns
target_cols = [x for x in sample_df.columns if 'responder' in x]
columns_to_drop = ['date_id', 'time_id', 'symbol_id', 'partition_id'] + target_cols

#
train_df = sample_df
target = 'responder_6'
train_cols = list(set(columns) - set(target))
X = train_df[train_cols]
y = train_df[target]
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(8000, 93) (2000, 93) (8000,) (2000,)


In [5]:
test_df = sample_df.to_pandas()

preproc = pipeline.Pipeline(
    steps=[
        ('drop_columns', ColumnTransformer(transformers=[('column_dropper', 'drop', columns_to_drop)], remainder='passthrough')),
        ('impute', SimpleImputer(strategy='mean')),
        ('min_max', preprocessing.MinMaxScaler()),
        ('norm', preprocessing.StandardScaler()),
    ]
)
X_train_trf = preproc.fit_transform(X_train)
X_test_trf = preproc.transform(X_test)

/usr/local/Caskroom/miniconda/base/envs/autocio/lib/python3.11/site-packages/sklearn/impute/_base.py:597: UserWarning: Skipping features without any observed values: [ 3  5 21 25 30 45 50 58 67]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/usr/local/Caskroom/miniconda/base/envs/autocio/lib/python3.11/site-packages/sklearn/impute/_base.py:597: UserWarning: Skipping features without any observed values: [ 3  5 21 25 30 45 50 58 67]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


In [6]:
model = linear_model.Lasso(alpha=0.1)
model.fit(X=X_train_trf, y=y_train)
y_pred = model.predict(X_test_trf)
print('MSE:', mean_squared_error(y_test, y_pred))
pl.DataFrame({'test':y_test, 'pred':y_pred})

MSE: 0.41272905


test,pred
f32,f32
-0.025232,-0.000961
-0.078556,-0.000961
-0.391851,-0.000961
-0.0157,-0.000961
0.434426,-0.000961
…,…
-0.70946,-0.000961
-0.229555,-0.000961
-0.11572,-0.000961


In [8]:
model = linear_model.Ridge()
model.fit(X=X_train_trf, y=y_train)
y_pred = model.predict(X_test_trf)
print('MSE:', mean_squared_error(y_test, y_pred))
pl.DataFrame({'test':y_test, 'pred':y_pred})

MSE: 0.36161366


test,pred
f32,f32
-0.025232,-0.198202
-0.078556,0.133886
-0.391851,0.264516
-0.0157,-0.16878
0.434426,0.12673
…,…
-0.70946,0.44443
-0.229555,0.089068
-0.11572,-0.537979


In [7]:
model = SVR(C=1.0, epsilon=0.2)
model.fit(X=X_train_trf, y=y_train)
y_pred = model.predict(X_test_trf)
print('MSE:', mean_squared_error(y_test, y_pred))
pl.DataFrame({'test':y_test, 'pred':y_pred})

MSE: 0.17014073427999715


test,pred
f32,f64
-0.025232,-0.042458
-0.078556,0.048043
-0.391851,-0.302542
-0.0157,-0.22303
0.434426,0.062831
…,…
-0.70946,-0.501093
-0.229555,-0.17808
-0.11572,-0.579353


In [ ]:
# test path
input_paths = 'inputs/test.parquet/*/*.parquet'
test_df = pl.scan_parquet()